In [1]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint 
import json

# Import API key
from config import google_api

#configure gmaps with api key 
gmaps.configure(api_key=google_api)



In [2]:
# loading csv from part 1 
weatherdf = pd.read_csv('Weather Data.csv')
weatherdf.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,George Town,5.4112,100.3354,77.90,86,20,3.44,MY,1629674739
1,1,Kapaa,22.0752,-159.3190,88.12,64,20,5.01,US,1629674740
2,2,Punta Arenas,-53.1500,-70.9167,37.51,93,90,13.80,CL,1629674636
3,3,Ponta do Sol,32.6667,-17.1000,72.72,78,0,3.22,PT,1629674744
4,4,Independence,39.0911,-94.4155,88.95,68,20,1.99,US,1629674488


In [5]:
#configure gmaps
fig = gmaps.figure()

locations = weatherdf[['Lat','Lng']]
humidity = weatherdf['Humidity']

heatmap_layer = gmaps.heatmap_layer(locations)
fig.add_layer(heatmap_layer)
fig


NameError: name 'zoom' is not defined

In [4]:
#ideal weather conditions 
maxtemp = weatherdf[weatherdf['Max Temp']<75]
mintemp = maxtemp[maxtemp['Max Temp']>65]
maxhum = mintemp[mintemp['Humidity']<50]
maxcloud = maxhum[maxhum['Cloudiness']<50]
vacay = maxcloud.dropna()
vacay

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
91,91,Manitouwadge,49.1215,-85.8403,66.92,42,17,0.87,CA,1629674815
142,142,Whitefish,48.4111,-114.3376,71.89,45,1,10.36,US,1629674633
167,167,Resavica,44.0408,21.5816,65.93,46,8,3.80,RS,1629674878
200,200,Dutlwe,-23.9833,23.9000,66.02,26,5,10.51,BW,1629674907
205,205,San Miguel de Tucumán,-26.8241,-65.2226,68.00,28,0,5.75,AR,1629674704
348,348,Kumul,42.8000,93.4500,65.37,24,0,16.40,CN,1629674988
432,432,Kushva,58.2906,59.7592,65.16,49,0,5.57,RU,1629675041
466,466,Erenhot,43.6475,111.9767,69.06,25,25,19.15,CN,1629675057
528,528,Magrath,49.4167,-112.8686,73.08,30,28,16.11,CA,1629675089


In [5]:

# params
params = {
    "radius": 5000,
    "types": "lodging",
    "key": google_api
}


for index, row in vacay.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()

    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        vacay.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


Retrieving Results for Index 91: Manitouwadge.
Closest hotel is Select Inn.
------------
Retrieving Results for Index 142: Whitefish.


C:\Users\Steph\anaconda3\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\Steph\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Closest hotel is Best Western Rocky Mountain Lodge.
------------
Retrieving Results for Index 167: Resavica.
Closest hotel is Hotel Đula.
------------
Retrieving Results for Index 200: Dutlwe.
Missing field/result... skipping.
------------
Retrieving Results for Index 205: San Miguel de Tucumán.
Closest hotel is HOTEL MIAMI.
------------
Retrieving Results for Index 348: Kumul.
Closest hotel is Laoyutang Resort.
------------
Retrieving Results for Index 432: Kushva.
Closest hotel is Hotel "Central" Kushva.
------------
Retrieving Results for Index 466: Erenhot.
Closest hotel is Erenhot Rurouni Youth Hostel.
------------
Retrieving Results for Index 528: Magrath.
Closest hotel is Lindenbaum Bed & Breakfast.
------------


In [60]:
vacay

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
91,91,Manitouwadge,49.1215,-85.8403,66.92,42,17,0.87,CA,1629674815,Select Inn
142,142,Whitefish,48.4111,-114.3376,71.89,45,1,10.36,US,1629674633,Best Western Rocky Mountain Lodge
167,167,Resavica,44.0408,21.5816,65.93,46,8,3.80,RS,1629674878,Hotel Đula
200,200,Dutlwe,-23.9833,23.9000,66.02,26,5,10.51,BW,1629674907,NaN
205,205,San Miguel de Tucumán,-26.8241,-65.2226,68.00,28,0,5.75,AR,1629674704,HOTEL MIAMI
348,348,Kumul,42.8000,93.4500,65.37,24,0,16.40,CN,1629674988,Laoyutang Resort
432,432,Kushva,58.2906,59.7592,65.16,49,0,5.57,RU,1629675041,"Hotel ""Central"" Kushva"
466,466,Erenhot,43.6475,111.9767,69.06,25,25,19.15,CN,1629675057,Erenhot Rurouni Youth Hostel
528,528,Magrath,49.4167,-112.8686,73.08,30,28,16.11,CA,1629675089,Lindenbaum Bed & Breakfast


In [6]:
# Assign the marker layer to a variable
locations = vacay[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in vacay.iterrows()]

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))